In [ ]:
import time
import json
import pickle
import re
import html
import sys
import gc

import gensim

from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, scale
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor

import pandas as pd
from pandas import DataFrame

from scipy.stats import spearmanr, pearsonr
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input
from keras.layers.wrappers import Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD, Adagrad
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from nltk import word_tokenize
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize.casual import TweetTokenizer

from IPython.display import display, HTML

In [ ]:
def browser_alert(message):
    display(HTML('<script type="text/javascript">alert("' + message + '");</script>'))
    
def browser_notify(message):
    display(HTML('<script type="text/javascript">var notification=new Notification("Jupyter Notification",{icon:"http://blog.jupyter.org/content/images/2015/02/jupyter-sq-text.png",body:"' + message + '"});</script>'))

In [ ]:
glove_twitter_path = "/Users/COMMANDER/Desktop/MScBA_Consultantcy_Project/GloVe/glove.twitter.27B/"
glove_path_6B = "/Users/COMMANDER/Desktop/MScBA_Consultantcy_Project/GloVe/glove.6B/"
glove_path_42B = "/Users/COMMANDER/Desktop/MScBA_Consultantcy_Project/GloVe/glove.42B.300d/"
glove_path_840B = "/Users/COMMANDER/Desktop/MScBA_Consultantcy_Project/GloVe/glove.840B.300d/"
glove_path_twitter = "/Users/COMMANDER/Desktop/MScBA_Consultantcy_Project/GloVe/glove.twitter.27B/"
wassa_home = "/Users/COMMANDER/Downloads/"

## Word2Vec + GloVe

In [ ]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r',encoding="utf8")
    model = {}
    num = 1
    for line in f:
        try:
            splitLine = line.split()
            word = splitLine[0]
            embedding = [float(val) for val in splitLine[1:]]
            model[word] = np.array(embedding)
            num += 1
        except Exception as e:
            print("Failed at line " + str(num))
    print("Done.",len(model)," words loaded!")
    return model

In [ ]:
# Google news pretrained vectors
wv_model_path = "GoogleNews-vectors-negative300.bin.gz"
wv_model = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path, binary=True, unicode_errors='ignore')

In [ ]:
# Twitter pretrained vectors
wv_model_path_1 = "word2vec_twitter_model.bin"
wv_model_1 = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path_1, binary=True, unicode_errors='ignore')

In [ ]:
wv_model_path_2 = glove_path_twitter + "glove.twitter.27B.200d.txt"
wv_model_2 = loadGloveModel(wv_model_path_2)

In [ ]:
wv_model_path_3 = glove_path_6B + "glove.6B.300d.txt"
wv_model_3 = loadGloveModel(wv_model_path_3)

In [ ]:
wv_model_path_4 = glove_path_42B + "glove.42B.300d.txt"
wv_model_4 = loadGloveModel(wv_model_path_4)

In [ ]:
wv_model_path_5 = glove_path_840B + "glove.840B.300d.txt"
wv_model_5 = loadGloveModel(wv_model_path_5)

In [ ]:
w2v_dimensions = len(wv_model['word'])
w2v_dimensions_1 = len(wv_model_1['word'])
w2v_dimensions_2 = len(wv_model_2['word'])
w2v_dimensions_3 = len(wv_model_3['word'])
w2v_dimensions_4 = len(wv_model_4['word'])
w2v_dimensions_5 = len(wv_model_5['word'])

print(w2v_dimensions, w2v_dimensions_1, w2v_dimensions_2, w2v_dimensions_3, w2v_dimensions_4, w2v_dimensions_5)

In [ ]:
def get_word2vec_embedding(word, model, dimensions):

    vec_rep = np.zeros(dimensions)
    if word in model:
        vec_rep = model[word]
    
    return vec_rep

In [ ]:
wnl = WordNetLemmatizer()

In [ ]:
from negate import NEGATE as neg
import nltk
nltk_sw = nltk.corpus.stopwords.words('english') #Number of nltk stop words: 179
nltk_sw_neg = [x for x in nltk_sw if x not in neg] #Number of nltk stop words without negating words: 158

In [ ]:
text="Applê isn't looking at buying U.K. startup for $1 billion. They still 600,20 4.5 + 423 look around."

In [ ]:
class Tweet(object):

    def __init__(self, id, text, emotion, intensity):
        self.id = id
        self.text = text
        self.emotion = emotion
        self.intensity = intensity

    def __repr__(self):
        return \
            "id: " + self.id + \
            ", text: " + self.text + \
            ", emotion: " + self.emotion + \
            ", intensity: " + self.intensity

In [ ]:
non_linear_factor = PolynomialFeatures(3)

In [ ]:
#emotion = "anger"

In [ ]:
#training_data_file_path = \
#    wassa_home + "corpora/multigenre/" + \
#    emotion + "-ratings-0to1.train.txt"
#word_embeddings_path = wassa_home + "embeddings/" + emotion + "-word-embeddings.pkl"

Feature Extraction Snippets

Emoji Intensity (Thanks to Roger, it works!)¶

In [ ]:
with open('/Users/COMMANDER/Downloads/lexicons/index_emoji.json', encoding="utf8") as emoji_file:
    emoji_list = json.load(emoji_file)
    
emoji_dict = dict()

for emoji in emoji_list:
    emoji_dict[emoji["emoji"]] = (emoji["name"], emoji["polarity"])

In [ ]:
def get_emoji_intensity(word):
    
    score = 0.0
    if word in emoji_dict.keys():
        score = float(emoji_dict[word][1])
    
    vec_rep = np.array([score])
    
    
    return non_linear_factor.fit_transform([vec_rep])[0]

In [ ]:
def lemm_word(word):
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import wordnet
    lemmatizer = WordNetLemmatizer()
    # first assuem it is a noun
    lem_token = lemmatizer.lemmatize(word, pos=wordnet.NOUN)
    # otherwise check for verb
    if lem_token == word:
        lem_token = lemmatizer.lemmatize(word, pos=wordnet.VERB)
    # otherwise check for adjektive
    if lem_token == word:
        lem_token = lemmatizer.lemmatize(word, pos=wordnet.ADJ)
    # otherwiese check for adverb
    if lem_token == word:
        lem_token = lemmatizer.lemmatize(word, pos=wordnet.ADV)
    return lem_token

In [ ]:
affect_intensity_file_path = \
    wassa_home + \
    "lexicons/NRC-AffectIntensity-Lexicon.txt"
    
def get_word_affect_intensity_dict(emotion):
    word_intensities = dict()

    with open(affect_intensity_file_path) as affect_intensity_file:
        for line in affect_intensity_file:
            word_int_array = line.replace("\n", "").split("\t")

            if (word_int_array[2] == emotion):
                word_intensities[word_int_array[0]] = float(word_int_array[1])

    return word_intensities

In [ ]:
#word_intensities = get_word_affect_intensity_dict(emotion)
word_intensities_anger = get_word_affect_intensity_dict('anger')
word_intensities_fear = get_word_affect_intensity_dict('fear')
word_intensities_joy = get_word_affect_intensity_dict('joy')
word_intensities_sadness = get_word_affect_intensity_dict('sadness')

In [ ]:
def get_emo_int_vector(word):
    
    a_score = 0.0
    f_score = 0.0
    j_score = 0.0
    s_score = 0.0

    if word in word_intensities_anger.keys():
        a_score = float(word_intensities_anger[word])
    else:
        word_lemm = lemm_word(word)
        if word_lemm in word_intensities_anger.keys():
            a_score = float(word_intensities_anger[word])

    if word in word_intensities_fear.keys():
        f_score = float(word_intensities_fear[word])
    else:
        word_lemm = lemm_word(word)
        if word_lemm in word_intensities_fear.keys():
            f_score = float(word_intensities_fear[word])

    if word in word_intensities_joy.keys():
        j_score = float(word_intensities_joy[word])
     else:
        word_lemm = lemm_word(word)
        if word_lemm in word_intensities_joy.keys():
            j_score = float(word_intensities_joy[word])

    if word in word_intensities_sadness.keys():
        s_score = float(word_intensities_sadness[word])
     else:
        word_lemm = lemm_word(word)
        if word_lemm in word_intensities_sadness.keys():
            s_score = float(word_intensities_sadness[word])

    vec_rep = np.array([a_score, f_score, j_score, s_score])
    
    return non_linear_factor.fit_transform([vec_rep])[0]

## SentiWordNet 

In [ ]:
def get_sentiwordnetscore(word):
    
    vec_rep = np.zeros(2)
    
    synsetlist = list(swn.senti_synsets(word))

    if synsetlist:
        vec_rep[0] = synsetlist[0].pos_score()
        vec_rep[1] = synsetlist[0].neg_score()

    return non_linear_factor.fit_transform([vec_rep])[0]

## Sentiment Emotion Presence Lexicon

In [ ]:
sentiment_emotion_lex_file_path = \
    wassa_home + \
    "Lexikon/NRC-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/" + \
    "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

def get_affect_presence_list(emotion):
    word_list = list()
    
    with open(sentiment_emotion_lex_file_path) as sentiment_emotion_lex_file:
        for line in sentiment_emotion_lex_file:
            word_array = line.replace("\n", "").split("\t")


            if (word_array[1] == emotion and word_array[2] == '1'):
                word_list.append(word_array[0])
    return word_list 

In [ ]:
#sentiment_emotion_lex_word_list = get_affect_presence_list(emotion)
sentiment_emotion_lex_word_list_anger = get_affect_presence_list("anger")
sentiment_emotion_lex_word_list_fear = get_affect_presence_list("fear")
sentiment_emotion_lex_word_list_joy = get_affect_presence_list("joy")
sentiment_emotion_lex_word_list_sadness = get_affect_presence_list("sadness")

In [ ]:
def get_sentiment_emotion_feature(word):
    
    a_score = 0.0
    f_score = 0.0
    j_score = 0.0
    s_score = 0.0

    if word in sentiment_emotion_lex_word_list_anger:
        a_score = 1.0
    else:
        word_lemm = lemm_word(word)
        if word_lemm in sentiment_emotion_lex_word_list_anger:
            a_score = 1.0

    if word in sentiment_emotion_lex_word_list_fear:
        f_score = 1.0
    else:
        word_lemm = lemm_word(word)
        if word_lemm in sentiment_emotion_lex_word_list_fear:
            f_score = 1.0

    if word in sentiment_emotion_lex_word_list_joy:
        j_score = 1.0
    else:
        word_lemm = lemm_word(word)
        if word_lemm in sentiment_emotion_lex_word_list_joy:
            j_score = 1.0

    if word in sentiment_emotion_lex_word_list_sadness:
        s_score = 1.0
    else:
        word_lemm = lemm_word(word)
        if word_lemm in sentiment_emotion_lex_word_list_sadness:
            s_score = 1.0

    vec_rep = np.array([a_score, f_score, j_score, s_score])
    
    return non_linear_factor.fit_transform([vec_rep])[0]

## Hashtag Emotion Intensity

In [ ]:
hashtag_emotion_lex_file_path = \
    wassa_home + \
    "Lexikon/NRC-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/" + \
    "NRC-Hashtag-Emotion-Lexicon-v0.2.txt"
    
def get_hashtag_emotion_intensity(emotion):
    hastag_intensities = dict()
    
    with open(hashtag_emotion_lex_file_path) as hashtag_emotion_lex_file:
        for line in hashtag_emotion_lex_file:
            word_array = line.replace("\n", "").split("\t")
            if (word_array[0] == emotion):
                hastag_intensities[word_array[1]] = float(word_array[2])
                
    return hastag_intensities

In [ ]:
#hashtag_emotion_intensities = get_hashtag_emotion_intensity(emotion)
hashtag_emotion_intensities_anger = get_hashtag_emotion_intensity('anger')
hashtag_emotion_intensities_fear = get_hashtag_emotion_intensity('fear')
hashtag_emotion_intensities_joy = get_hashtag_emotion_intensity('joy')
hashtag_emotion_intensities_sadness = get_hashtag_emotion_intensity('sadness')

In [ ]:
def get_hashtag_emotion_vector(word):
    
    a_score = 0.0
    f_score = 0.0
    j_score = 0.0
    s_score = 0.0
    
    if word in hashtag_emotion_intensities_anger.keys():
        a_score = float(hashtag_emotion_intensities_anger[word])
    else:
        word_lemm = lemm_word(word)
        if word_lemm in hashtag_emotion_intensities_anger.keys():
            a_score = float(hashtag_emotion_intensities_anger[word_lemm])

    if word in hashtag_emotion_intensities_fear.keys():
        f_score = float(hashtag_emotion_intensities_fear[word])
    else:
        word_lemm = lemm_word(word)
        if word_lemm in hashtag_emotion_intensities_fear.keys():
            f_score = float(hashtag_emotion_intensities_fear[word_lemm])

    if word in hashtag_emotion_intensities_joy.keys():
        j_score = float(hashtag_emotion_intensities_joy[word])
    else:
        word_lemm = lemm_word(word)
        if word_lemm in hashtag_emotion_intensities_joy.keys():
            j_score = float(hashtag_emotion_intensities_joy[word_lemm])

    if word in hashtag_emotion_intensities_sadness.keys():
        s_score = float(hashtag_emotion_intensities_sadness[word])
    else:
        word_lemm = lemm_word(word)
        if word_lemm in hashtag_emotion_intensities_sadness.keys():
            s_score = float(hashtag_emotion_intensities_sadness[word_lemm])
        
    vec_rep = np.array([a_score, f_score, j_score, s_score])
            
    return non_linear_factor.fit_transform([vec_rep])[0]

## Emoticon Sentiment Lexicon

In [ ]:
emoticon_lexicon_unigrams_file_path = \
    wassa_home + \
    "lexicons/Emoticon-unigrams.txt"
emoticon_lexicon_bigrams_file_path = \
    wassa_home + \
    "lexicons/Emoticon-bigrams.txt"
    
emoticon_lexicon_unigrams = dict()
emoticon_lexicon_bigrams = dict()

def get_emoticon_lexicon_unigram_dict():
    with open(emoticon_lexicon_unigrams_file_path) as emoticon_lexicon_file:
        for line in emoticon_lexicon_file:
            word_array = line.replace("\n", "").split("\t")
            emoticon_lexicon_unigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return emoticon_lexicon_unigrams

def get_emoticon_lexicon_bigram_dict():
    with open(emoticon_lexicon_bigrams_file_path) as emoticon_lexicon_file:
        for line in emoticon_lexicon_file:
            word_array = line.replace("\n", "").split("\t")
            emoticon_lexicon_bigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return emoticon_lexicon_bigrams

In [ ]:
emoticon_lexicon_unigram_dict = get_emoticon_lexicon_unigram_dict()

In [ ]:
emoticon_lexicon_bigram_dict = get_emoticon_lexicon_bigram_dict()

In [ ]:
def get_unigram_sentiment_emoticon_lexicon_vector(word):
    
    vec_rep = np.zeros(3)
    if word in emoticon_lexicon_unigram_dict.keys():
        vec_rep = emoticon_lexicon_unigram_dict[word]
    else:
        word_lemm = lemm_word(word)
        if word_lemm in emoticon_lexicon_unigram_dict.keys():
            vec_rep = emoticon_lexicon_unigram_dict[word_lemm]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

def get_bigram_sentiment_emoticon_lexicon_vector(word):
    
    vec_rep = np.zeros(3)
    if word in emoticon_lexicon_bigram_dict.keys():
        vec_rep = emoticon_lexicon_bigram_dict[word]
    else:
        word_lemm = lemm_word(word)
        if word_lemm in emoticon_lexicon_bigram_dict.keys():
            vec_rep = emoticon_lexicon_bigram_dict[word_lemm]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

## Emoticon Sentiment Aff-Neg Lexicon

In [ ]:
emoticon_afflex_unigrams_file_path = \
    wassa_home + \
    "lexicons/Emoticon-AFFLEX-NEGLEX-unigrams.txt"

emoticon_afflex_bigrams_file_path = \
    wassa_home + \
    "lexicons/Emoticon-AFFLEX-NEGLEX-bigrams.txt"
    
emoticon_afflex_unigrams = dict()
emoticon_afflex_bigrams = dict()

def get_emoticon_afflex_unigram_dict():
    with open(emoticon_afflex_unigrams_file_path) as emoticon_lexicon_file:
        for line in emoticon_lexicon_file:
            word_array = line.replace("\n", "").split("\t")
            emoticon_afflex_unigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return emoticon_afflex_unigrams

def get_emoticon_afflex_bigram_dict():
    with open(emoticon_afflex_bigrams_file_path) as emoticon_lexicon_file:
        for line in emoticon_lexicon_file:
            word_array = line.replace("\n", "").split("\t")
            emoticon_afflex_bigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return emoticon_afflex_bigrams

In [ ]:
emoticon_afflex_unigram_dict = get_emoticon_afflex_unigram_dict()

In [ ]:
emoticon_afflex_bigram_dict = get_emoticon_afflex_bigram_dict()

In [ ]:
def get_unigram_sentiment_emoticon_afflex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in emoticon_afflex_unigram_dict.keys():
        vec_rep = emoticon_afflex_unigram_dict[word]
    else:
        word_lemm = lemm_word(word)
        if word_lemm in emoticon_afflex_unigram_dict.keys():
            vec_rep = emoticon_afflex_unigram_dict[word_lemm]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

def get_bigram_sentiment_emoticon_afflex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in emoticon_afflex_bigram_dict.keys():
        vec_rep = emoticon_afflex_bigram_dict[word]
    else:
        word_lemm = lemm_word(word)
        if word_lemm in emoticon_afflex_bigram_dict.keys():
            vec_rep = emoticon_afflex_bigram_dict[word_lemm]
    
    return non_linear_factor.fit_transform([vec_rep])[0]

## Hashtag Sentiment Aff-Neg Lexicon

In [ ]:
hashtag_affneglex_unigrams_file_path = \
    wassa_home + \
    "lexicons/HS-AFFLEX-NEGLEX-unigrams.txt"
hashtag_affneglex_bigrams_file_path = \
    wassa_home + \
    "lexicons/HS-AFFLEX-NEGLEX-bigrams.txt"
    
hashtag_affneglex_unigrams = dict()
hashtag_affneglex_bigrams = dict()

def get_hashtag_affneglex_unigram_dict():
    with open(hashtag_affneglex_unigrams_file_path) as hashtag_sent_lex_file:
        for line in hashtag_sent_lex_file:
            word_array = line.replace("\n", "").split("\t")
            hashtag_affneglex_unigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return hashtag_affneglex_unigrams

def get_hashtag_affneglex_bigram_dict():
    with open(hashtag_affneglex_bigrams_file_path) as hashtag_sent_lex_file:
        for line in hashtag_sent_lex_file:
            word_array = line.replace("\n", "").split("\t")
            hashtag_affneglex_bigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])

    return hashtag_affneglex_bigrams

In [ ]:
hashtag_affneglex_unigram_dict = get_hashtag_affneglex_unigram_dict()

In [ ]:
hashtag_affneglex_bigram_dict = get_hashtag_affneglex_bigram_dict()

In [ ]:
def get_unigram_sentiment_hashtag_affneglex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in hashtag_affneglex_unigram_dict.keys():
        vec_rep = hashtag_affneglex_unigram_dict[word]
    else:
        word_lemm = lemm_word(word)
        if word_lemm in hashtag_affneglex_unigram_dict.keys():
            vec_rep = hashtag_affneglex_unigram_dict[word_lemm]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

def get_bigram_sentiment_hashtag_affneglex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in hashtag_affneglex_bigram_dict.keys():
        vec_rep = hashtag_affneglex_bigram_dict[word]
    else:
        word_lemm = lemm_word(word)
        if word_lemm in hashtag_affneglex_bigram_dict.keys():
            vec_rep = hashtag_affneglex_bigram_dict[word_lemm]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

## Hashtag Sentiment Lexicon

In [ ]:
hash_sent_lex_unigrams_file_path = \
    wassa_home + \
    "lexicons/HS-unigrams.txt"
hash_sent_lex_bigrams_file_path = \
    wassa_home + \
    "lexicons/HS-bigrams.txt"

def get_hash_sent_lex_unigram_dict():
    
    hash_sent_lex_unigrams = dict()
    with open(hash_sent_lex_unigrams_file_path) as hash_sent_lex_file:
        for line in hash_sent_lex_file:
            word_array = line.replace("\n", "").split("\t")
            if clean_str(word_array[0]):
                hash_sent_lex_unigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return hash_sent_lex_unigrams

def get_hash_sent_lex_bigram_dict():

    hash_sent_lex_bigrams = dict()
    with open(hash_sent_lex_bigrams_file_path) as hash_sent_lex_file:
        for line in hash_sent_lex_file:
            word_array = line.replace("\n", "").split("\t")
            if clean_str(word_array[0]):
                hash_sent_lex_bigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return hash_sent_lex_bigrams

In [ ]:
hash_sent_lex_unigram_dict = get_hash_sent_lex_unigram_dict()

In [ ]:
hash_sent_lex_bigram_dict = get_hash_sent_lex_bigram_dict()

In [ ]:
def get_unigram_sentiment_hash_sent_lex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in hash_sent_lex_unigram_dict.keys():
        vec_rep = hash_sent_lex_unigram_dict[word]
    else:
        word_lemm = lemm_word(word)
        if word_lemm in hash_sent_lex_unigram_dict.keys():
            vec_rep = hash_sent_lex_unigram_dict[word_lemm]
        
    return non_linear_factor.fit_transform([vec_rep])[0]


def get_bigram_sentiment_hash_sent_lex_vector(word):

    vec_rep = np.zeros(3)
    if word in hash_sent_lex_bigram_dict.keys():
        vec_rep = hash_sent_lex_bigram_dict[word]
    else:
        word_lemm = lemm_word(word)
        if word_lemm in hash_sent_lex_bigram_dict.keys():
            vec_rep = hash_sent_lex_bigram_dict[word_lemm]
            
    return non_linear_factor.fit_transform([vec_rep])[0]

## Depeche Mood (There is DepecheMood V2, try it)

In [ ]:
depeche_mood_file_path = \
    wassa_home + \
    "lexicons/DepecheMood_V1.0/DepecheMood_normfreq2.txt"

In [ ]:
def get_depeche_vector_dict():
    depeche_vector_dict = dict()
    with open(depeche_mood_file_path) as depeche_mood_file:
        for line in depeche_mood_file:
            word_array = line.replace("\n", "").split("\t")
            depeche_vector_dict[word_array[0].split("#")[0]] = np.array([float(val) for val in word_array[1:]])
    
    return depeche_vector_dict

In [ ]:
depeche_vector_dict = get_depeche_vector_dict()

In [ ]:
def get_depeche_mood_vector(word):
    
    vec_rep = np.zeros(8)
    if word in depeche_vector_dict.keys():
        vec_rep = np.array(depeche_vector_dict[word])
    else:
        word_lemm = lemm_word(word)
        if word_lemm in depeche_vector_dict.keys():
            vec_rep = np.array(depeche_vector_dict[word_lemm])

    return non_linear_factor.fit_transform([vec_rep])[0]

# Reading & Vectorizing Data

In [ ]:
def is_active_vector_method(string):
    return int(string)

def learn_unigram_word_embedding(word):
    
    word_feature_embedding_dict = dict()
    
    '''Pre-trained Word embeddings'''
    index = 0
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model, w2v_dimensions)

    index = 1
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_1, w2v_dimensions_1)

    index = 2
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_2, w2v_dimensions_2)

    index = 3
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_3, w2v_dimensions_3)

    index = 4
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_4, w2v_dimensions_4)

    index = 5
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_5, w2v_dimensions_5)

    '''NRC Emotion Intensity Lexicon'''
    index = 6
    word_feature_embedding_dict[index] = get_emo_int_vector(word)

    '''WordNet'''
    index = 7
    word_feature_embedding_dict[index] = get_sentiwordnetscore(word)

    '''NRC Sentiment Lexica'''
    index = 8
    word_feature_embedding_dict[index] = get_sentiment_emotion_feature(word)

    index = 9
    word_feature_embedding_dict[index] = get_unigram_sentiment_emoticon_lexicon_vector(word)

    index = 10
    word_feature_embedding_dict[index] = get_unigram_sentiment_emoticon_afflex_vector(word)

    '''NRC Hashtag Lexica'''
    index = 11
    word_feature_embedding_dict[index] = get_hashtag_emotion_vector(word)

    index = 12
    word_feature_embedding_dict[index] = get_unigram_sentiment_hash_sent_lex_vector(word)

    index = 13
    word_feature_embedding_dict[index] = get_unigram_sentiment_hashtag_affneglex_vector(word)

    '''Emoji Polarities'''
    index = 14
    word_feature_embedding_dict[index] = get_emoji_intensity(word)
    
    '''Depeche Mood'''
    index = 15
    word_feature_embedding_dict[index] = get_depeche_mood_vector(word)

    return word_feature_embedding_dict


def learn_bigram_word_embedding(word):
    
    word_feature_embedding_dict = dict()
    
    '''NRC Sentiment Lexica'''

    index = 0
    word_feature_embedding_dict[index] = get_bigram_sentiment_emoticon_lexicon_vector(word)

    index = 1
    word_feature_embedding_dict[index] = get_bigram_sentiment_emoticon_afflex_vector(word)

    '''NRC Hashtag Lexica'''
    index = 2
    word_feature_embedding_dict[index] = get_bigram_sentiment_hash_sent_lex_vector(word)

    index = 3
    word_feature_embedding_dict[index] = get_bigram_sentiment_hashtag_affneglex_vector(word)

    return word_feature_embedding_dict

In [ ]:
def get_unigram_embedding(word, word_embedding_dict, bin_string):
    
    word_feature_embedding_dict = word_embedding_dict[word]
    final_embedding = np.array([])
    
    for i in range(16):
        if is_active_vector_method(bin_string[i]):
            final_embedding = np.append(final_embedding, word_feature_embedding_dict[i])
    
    return final_embedding

def get_bigram_embedding(bigram, word_embedding_dict, bin_string):
    
    word_feature_embedding_dict = word_embedding_dict[word]
    final_embedding = np.array([])
    
    for i in range(4):
        if is_active_vector_method(bin_string[i]):
            final_embedding = np.append(final_embedding, word_feature_embedding_dict[i])
    
    return final_embedding

In [ ]:
unigram_feature_string = "1111111111111111"
bigram_feature_string = "1111"

In [ ]:
def build_word_embeddings(tweets):
    
    max_tweet_length = -1
    word_embedding_dict = dict()

    for tweet in tweets:
        if len(tweet) > max_tweet_length:
            max_tweet_length = len(tweet)

        for token in tweet:
            if token not in word_embedding_dict.keys():
                word_embedding_dict[token] = learn_unigram_word_embedding(token)
                
    return word_embedding_dict, max_tweet_length

In [ ]:
def get_word_tokens(tokenizer, sequences):
    result = list()
    for s in sequences:
        texttokens = list()
        for token in s:
            texttokens.append(tokenizer.index_word[token])
        result.append(list(texttokens))
    return result

In [ ]:
from corpora_utils import CorporaHelper, CorporaProperties
corpora_helper = CorporaHelper("corpora/multigenre.csv", separator = ';')

corpora_helper.remove_emotion("other")
corpora_helper.remove_emotion("noemotion")
corpora_helper.remove_emotion("trust")
corpora_helper.remove_emotion("anticipation")
corpora_helper.remove_emotion("surprise")
corpora_helper.remove_emotion("disgust")

corpora_helper.translate_urls()
corpora_helper.translate_emoticons()
corpora_helper.translate_emojis()
corpora_helper.translate_email()
#corpora_helper.translate_mention()
corpora_helper.translate_html_tags()
#corpora_helper.translate_camel_case()
corpora_helper.translate_underscore()

corpora_helper.translate_string('-LRB-','(')
corpora_helper.translate_string('-RRB-',')')
corpora_helper.translate_string('`',"'") # ` to '
corpora_helper.translate_string("''",'"') # double '' to "
corpora_helper.translate_contractions()
corpora_helper.translate_string("'","") # remove '
corpora_helper.translate_string("\\n"," ") # replace new lines with space

#corpora_helper.spell_correction()
corpora_helper.add_space_at_special_chars()
corpora_helper.translate_to_lower()

# tokenize the cleaned corpora
corpora = list()
texts = list()
for corpus_id, corpus in corpora_helper.get_data().iterrows():
    texts.append(corpus[CorporaProperties.CLEANED_CORPUS.value])

tokenizer = Tokenizer(filters = '')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
# Create word tokenlist
corpora = get_word_tokens(tokenizer,sequences)


In [ ]:
embedding_info = build_word_embeddings(corpora)

In [ ]:
pickle.dump(embedding_info, open("multigenre_embedding.pkl", "wb"))

In [ ]:
browser_notify("Persisted to disk")

In [ ]:
from corpora_utils import CorporaHelper, CorporaProperties
corpora_helper = CorporaHelper("corpora/twitter_all.csv", separator = '\t')
corpora_helper.remove_column('intensity')
corpora_helper.add_domain('twitter')

corpora_helper.translate_urls()
corpora_helper.translate_emoticons()
corpora_helper.translate_emojis()
corpora_helper.translate_email()
#corpora_helper.translate_mention()
corpora_helper.translate_html_tags()
#corpora_helper.translate_camel_case()
corpora_helper.translate_underscore()

corpora_helper.translate_string('-LRB-','(')
corpora_helper.translate_string('-RRB-',')')
corpora_helper.translate_string('`',"'") # ` to '
corpora_helper.translate_string("''",'"') # double '' to "
corpora_helper.translate_contractions()
corpora_helper.translate_string("'","") # remove '
corpora_helper.translate_string("\\n"," ") # replace new lines with space

#corpora_helper.spell_correction()
corpora_helper.add_space_at_special_chars()
corpora_helper.translate_to_lower()

# tokenize the cleaned corpora
corpora = list()
texts = list()
for corpus_id, corpus in corpora_helper.get_data().iterrows():
    texts.append(corpus[CorporaProperties.CLEANED_CORPUS.value])

tokenizer = Tokenizer(filters = '')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
# Create word tokenlist
twitter_corpora = get_word_tokens(tokenizer,sequences)

In [ ]:
twitter_embedding_info = build_word_embeddings(twitter_corpora)

In [ ]:
pickle.dump(twitter_embedding_info, open("twitter_embedding.pkl", "wb"))